In [1]:
import pandas as pd 
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly_express as px
from geopy.point import Point
from tqdm import tqdm
#from pandas_profiling import ProfileReport

## Import df from src 
filename = 'src/data_100000.csv'
df = pd.read_csv(filename)
#Renaming col
df = df.rename(columns={"vehicle_type_code1": "vehicle_type_code_1", "vehicle_type_code2": "vehicle_type_code_2"})
df.head()
#profile = ProfileReport(df, title="Pandas Profiling Report")

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2017-04-18T00:00:00.000,23:10,STATEN ISLAND,10312.0,40.536728,-74.193344,"(40.536728, -74.193344)",NaN,NaN,243 DARLINGTON AVENUE,...,Unspecified,NaN,NaN,NaN,3654181,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1,2017-05-06T00:00:00.000,13:00,BRONX,10472.0,40.829052,-73.850380,"(40.829052, -73.85038)",CASTLE HILL AVENUE,BLACKROCK AVENUE,NaN,...,NaN,NaN,NaN,NaN,3665311,Sedan,NaN,NaN,NaN,NaN
2,2017-04-27T00:00:00.000,17:15,QUEENS,11420.0,40.677303,-73.804565,"(40.677303, -73.804565)",135 STREET,FOCH BOULEVARD,NaN,...,Unspecified,NaN,NaN,NaN,3658491,Sedan,Sedan,NaN,NaN,NaN
3,2017-05-09T00:00:00.000,20:10,NaN,NaN,40.624958,-74.145775,"(40.624958, -74.145775)",FOREST AVENUE,RICHMOND AVENUE,NaN,...,Unspecified,Unspecified,NaN,NaN,3666554,Motorcycle,Sedan,Bus,NaN,NaN
4,2017-04-18T00:00:00.000,14:00,BRONX,10456.0,40.828846,-73.903120,"(40.828846, -73.90312)",NaN,NaN,1167 BOSTON ROAD,...,Unspecified,NaN,NaN,NaN,3653269,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [2]:
## Check for missing,nul,NaN values
print("There are {} rows of data".format(len(df)))
nul_data = df.isnull().sum()
## setting to false missing data
nul_data
df.vehicle_type_code_1.unique()

There are 100000 rows of data


array(['Station Wagon/Sport Utility Vehicle', 'Sedan', 'Motorcycle',
       'Taxi', 'Pick-up Truck', 'Ambulance', 'Tractor Truck Diesel',
       'Bus', nan, 'Box Truck', 'Bike', 'deliv', 'Van', 'tower', 'PAS',
       'Tanker', 'Flat Bed', 'Convertible', 'Motorscooter', 'Moped',
       'e-350', 'PASSENGER VEHICLE', 'RV',
       'SPORT UTILITY / STATION WAGON', 'Garbage or Refuse',
       'Chassis Cab', 'self', 'Concrete Mixer', 'Tractor Truck Gasoline',
       '2 dr sedan', 'Tow Truck / Wrecker', 'Beverage Truck', 'Flat Rack',
       'Carry All', 'Motorbike', 'REFG', 'van', '4 dr sedan',
       'Bulk Agriculture', 'PK', 'POSTA', 'TRACT', 'E-Bike', 'DELV',
       'AMB', 'VAN', 'MACK', 'Lift Boom', 'TAXI', '3-Door',
       'SMALL COM VEH(4 TIRES) ', 'winne', 'AMBUL', 'p/sh', 'Dump',
       'Refrigerated Van', 'BOBCA', 'School Bus', 'truck',
       'Armored Truck', 'UNKNO', 'TRUCK', 'WHITE', 'COM', '4D', 'box t',
       'E-Scooter', 'Mack', 'fire', 'UTIL', 'LADDER 34', 'moped', 'DUMP',
   

In [14]:
from textblob import TextBlob
import difflib
from fuzzywuzzy import fuzz
vehicle_type_code = ['vehicle_type_code_1','vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5']
type_of_car = ['Hatchback','Sedan','SUV','MUV','Crossover','Coupe','Convertible']
nb_accident_by_vehicle = df[['vehicle_type_code_1','vehicle_type_code_2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5']]
df[vehicle_type_code] = df[vehicle_type_code].fillna('Unspecified')

def fuzzy_replace(x, names,i):
    print(len(names))
    aliases = difflib.get_close_matches(x, names, len(names), 0.8)
    closest = pd.Series(aliases).mode()
    closest = aliases[0] if closest.empty else closest[0]
    df[i].replace(aliases, closest, True)
    
for i in df[vehicle_type_code]:
    df[i] = df[i].apply(lambda x: x.strip())
    df[i] = [w.lower() for w in df[i]]

for i in df[vehicle_type_code]: 
    print(i)
    for x in df[i]:
        print(x)
        fuzzy_replace(x,df[i].unique(),i)
        
df.vehicle_type_code_1.unique()

d
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspecified
17
unspe

array(['station wagon/sport utility vehicle', 'sedan', 'motorcycle',
       'taxi', 'pick up', 'abulance', 'tractor truck diesel', 'bs',
       'unspecified', 'box truck', 'bike', 'deliv', 'van', 'power', 'pas',
       'tanker', 'flat bed', 'convertible', 'motor scoo', 'moped',
       'e-350', 'passenger vehicle', 'rv',
       'sport utility / station wagon', 'garbage or refuse',
       'chassis cab', 'self', 'concrete mixer', 'tractor truck gasoline',
       '2 dr sedan', 'tow truck / wrecker', 'beverage truck', 'flat rack',
       'carry all', 'motorbike', 'refg', 'bulk agriculture', 'pk',
       'posta', 'trac', 'amb', 'mack', 'lift boom', '3-door',
       'small com veh(4 tires)', 'wineb', 'ambul', 'p/sh', 'dump',
       'refrigerated van', 'bobca', 'school bus', 'truck',
       'armored truck', 'unk', 'white', 'com', '4d', 'box t', 'e-scooter',
       'fire', 'util', 'ladder 34', 'van camper', 'e-sco', 'golf',
       'stake or rack', 'e bik', 'const', 'ltr', 'schoo', 'fork',
     

In [17]:
contributing_factor = ['contributing_factor_vehicle_1','contributing_factor_vehicle_2','contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5']
nb_accident_by_factor = df[['contributing_factor_vehicle_1','contributing_factor_vehicle_2','contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5']].apply(pd.value_counts)
df[contributing_factor]= df[contributing_factor].fillna('Unspecidifed')

for i in df[contributing_factor]:
    df[i] = df[i].apply(lambda x: x.strip())
    df[i] = [w.lower() for w in df[i]]

for i in df[contributing_factor]: 
    print(i)
    for x in df[i]:
        print(x)
        fuzzy_replace(x,df[i].unique(),i)
nb_accident_by_factor.index

ecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9
unspecidifed
9

Index(['Accelerator Defective', 'Aggressive Driving/Road Rage',
       'Alcohol Involvement', 'Animals Action', 'Backing Unsafely',
       'Brakes Defective', 'Cell Phone (hand-Held)', 'Cell Phone (hands-free)',
       'Driver Inattention/Distraction', 'Driver Inexperience',
       'Driverless/Runaway Vehicle', 'Drugs (illegal)', 'Eating or Drinking',
       'Failure to Keep Right', 'Failure to Yield Right-of-Way',
       'Fatigued/Drowsy', 'Fell Asleep', 'Following Too Closely', 'Glare',
       'Headlights Defective', 'Illnes', 'Lane Marking Improper/Inadequate',
       'Listening/Using Headphones', 'Lost Consciousness',
       'Obstruction/Debris', 'Other Electronic Device',
       'Other Lighting Defects', 'Other Vehicular', 'Outside Car Distraction',
       'Oversized Vehicle', 'Passenger Distraction', 'Passing Too Closely',
       'Passing or Lane Usage Improper', 'Pavement Defective',
       'Pavement Slippery',
       'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
    

In [7]:
df.to_csv('consolidated_data.csv')

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2017-04-18T00:00:00.000,23:10,STATEN ISLAND,10312.0,40.536728,-74.193344,"(40.536728, -74.193344)",NaN,NaN,243 DARLINGTON AVENUE,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,3654181,station wagon/sport utility vehicle,Unspecified,Unspecified,Unspecified,Unspecified
1,2017-05-06T00:00:00.000,13:00,BRONX,10472.0,40.829052,-73.850380,"(40.829052, -73.85038)",CASTLE HILL AVENUE,BLACKROCK AVENUE,NaN,...,Unspecidifed,Unspecidifed,Unspecidifed,Unspecidifed,3665311,sedan,Unspecified,Unspecified,Unspecified,Unspecified
2,2017-04-27T00:00:00.000,17:15,QUEENS,11420.0,40.677303,-73.804565,"(40.677303, -73.804565)",135 STREET,FOCH BOULEVARD,NaN,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,3658491,sedan,Sedan,Unspecified,Unspecified,Unspecified
3,2017-05-09T00:00:00.000,20:10,NaN,NaN,40.624958,-74.145775,"(40.624958, -74.145775)",FOREST AVENUE,RICHMOND AVENUE,NaN,...,Unspecified,Unspecified,Unspecidifed,Unspecidifed,3666554,motorcycle,Sedan,Bus,Unspecified,Unspecified
4,2017-04-18T00:00:00.000,14:00,BRONX,10456.0,40.828846,-73.903120,"(40.828846, -73.90312)",NaN,NaN,1167 BOSTON ROAD,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,3653269,sedan,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-11-20T00:00:00.000,15:00,BROOKLYN,11210.0,40.618893,-73.946420,"(40.618893, -73.94642)",NaN,NaN,1314 EAST 29 STREET,...,Unspecidifed,Unspecidifed,Unspecidifed,Unspecidifed,4244961,station wagon/sport utility vehicle,Unspecified,Unspecified,Unspecified,Unspecified
99996,2019-12-01T00:00:00.000,11:22,QUEENS,11367.0,40.723380,-73.814750,"(40.72338, -73.81475)",NaN,NaN,150-62 76 ROAD,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,4250093,station wagon/sport utility vehicle,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified
99997,2019-11-21T00:00:00.000,21:30,BROOKLYN,11249.0,40.710820,-73.968530,"(40.71082, -73.96853)",BROADWAY,KENT AVENUE,NaN,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,4245290,sedan,Box Truck,Unspecified,Unspecified,Unspecified
99998,2019-11-18T00:00:00.000,17:28,BROOKLYN,11234.0,40.631180,-73.928185,"(40.63118, -73.928185)",NaN,NaN,1695 UTICA AVENUE,...,Unspecified,Unspecidifed,Unspecidifed,Unspecidifed,4243646,sedan,Bus,Unspecified,Unspecified,Unspecified


['OTTO J MAYER', 'OTTO MAYER']
0    OTTO J MAYER
1      OTTO MAYER
dtype: object
['OTTO J MAYER', 'OTTO J MAYER']
0    OTTO J MAYER
dtype: object
['DANIEL J ROSEN']
0    DANIEL J ROSEN
dtype: object
['DANIEL ROSSY']
0    DANIEL ROSSY
dtype: object
['LISA CULLI', 'LISA CULLY']
0    LISA CULLI
1    LISA CULLY
dtype: object
['LISA CULLI', 'LISA CULLI']
0    LISA CULLI
dtype: object


,OWNER
0,OTTO J MAYER
1,OTTO J MAYER
2,DANIEL J ROSEN
3,DANIEL ROSSY
4,LISA CULLI
5,LISA CULLI


crash_date                          0
crash_time                          0
borough                             0
zip_code                            0
latitude                         8035
longitude                        8035
location                         8035
on_street_name                      0
off_street_name                     0
cross_street_name                   0
number_of_persons_injured           0
number_of_persons_killed            0
number_of_pedestrians_injured       0
number_of_pedestrians_killed        0
number_of_cyclist_injured           0
number_of_cyclist_killed            0
number_of_motorist_injured          0
number_of_motorist_killed           0
contributing_factor_vehicle_1       0
contributing_factor_vehicle_2       0
contributing_factor_vehicle_3       0
contributing_factor_vehicle_4       0
contributing_factor_vehicle_5       0
collision_id                        0
vehicle_type_code_1                 0
vehicle_type_code_2                 0
vehicle_type

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,adress
24,2018-02-17T00:00:00.000,14:30,MANHATTAN,10019.0,40.736576,-73.988543,NaN,WEST 263 STREET,WEST 262 STREET,450 W 57th St,...,Unspecidifed,Unspecidifed,Unspecidifed,3848714.0,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"WEST 263 STREET,MANHATTAN,United States of Ame..."
25,2019-12-04T00:00:00.000,15:30,STATEN ISLAND,10309.0,40.583456,-74.149605,NaN,WEST 263 STREET,WEST 262 STREET,715 sharrotts rd,...,Unspecidifed,Unspecidifed,Unspecidifed,4263561.0,Tractor Truck Diesel,Unspecified,Unspecified,Unspecified,Unspecified,"WEST 263 STREET,STATEN ISLAND,United States of..."
26,2017-09-12T00:00:00.000,17:59,STATEN ISLAND,10309.0,NaN,NaN,NaN,PARK ROW,WEST 262 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,3749038.0,Bus,Unspecified,Unspecified,Unspecified,Unspecified,"PARK ROW,STATEN ISLAND,United States of America"
27,2017-05-07T00:00:00.000,15:00,STATEN ISLAND,10309.0,40.583456,-74.149605,NaN,CENTER BOULEVARD,WEST 262 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,3668217.0,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified,"CENTER BOULEVARD,STATEN ISLAND,United States o..."
36,2017-04-25T00:00:00.000,19:30,STATEN ISLAND,10309.0,NaN,NaN,NaN,THROGS NECK EXPRESSWAY,HARDING AVENUE,False,...,Unspecidifed,Unspecidifed,Unspecidifed,3658008.0,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"THROGS NECK EXPRESSWAY,STATEN ISLAND,United St..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8032,NaN,NaN,NaN,NaN,40.761720,-73.801821,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8033,NaN,NaN,NaN,NaN,40.823524,-73.878668,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8034,NaN,NaN,NaN,NaN,40.728185,-73.928154,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,adress
91845,2020-03-28T00:00:00.000,18:50,BROOKLYN,11233.0,0.0,0.0,"(0.0, 0.0)",ATLANTIC AVENUE,SACKMAN STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4304383,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"ATLANTIC AVENUE,BROOKLYN,United States of Amer..."
70102,2020-06-16T00:00:00.000,8:11,QUEENS,11104.0,0.0,0.0,"(0.0, 0.0)",43 STREET,QUEENS BOULEVARD,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4320537,Station Wagon/Sport Utility Vehicle,Sedan,Unspecified,Unspecified,Unspecified,"43 STREET,QUEENS,United States of America"
49443,2020-09-29T00:00:00.000,3:20,BROOKLYN,11209.0,0.0,0.0,"(0.0, 0.0)",43 STREET,QUEENS BOULEVARD,339 BAY RIDGE PARKWAY,...,Unspecidifed,Unspecidifed,Unspecidifed,4352797,Sedan,Sedan,Unspecified,Unspecified,Unspecified,"43 STREET,BROOKLYN,United States of America"
49549,2020-09-13T00:00:00.000,14:25,BRONX,10468.0,0.0,0.0,"(0.0, 0.0)",EAST BEDFORD PARK BOULEVARD,VILLA AVENUE,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4347357,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified,"EAST BEDFORD PARK BOULEVARD,BRONX,United State..."
49584,2020-09-28T00:00:00.000,0:00,QUEENS,11412.0,0.0,0.0,"(0.0, 0.0)",115 AVENUE,200 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4352807,Sedan,Sedan,Sedan,Unspecified,Unspecified,"115 AVENUE,QUEENS,United States of America"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99935,2019-11-21T00:00:00.000,17:00,BRONX,10467.0,NaN,NaN,NaN,CROSS ISLAND PARKWAY,149 STREET,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4245413,Sedan,Unspecified,Unspecified,Unspecified,Unspecified,"CROSS ISLAND PARKWAY,BRONX,United States of Am..."
99944,2019-11-15T00:00:00.000,13:52,QUEENS,11373.0,NaN,NaN,NaN,HORACE HARDING EXPRESSWAY,QUEENS BOULEVARD,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4241598,Station Wagon/Sport Utility Vehicle,Box Truck,Unspecified,Unspecified,Unspecified,"HORACE HARDING EXPRESSWAY,QUEENS,United States..."
99965,2020-03-04T00:00:00.000,23:47,QUEENS,11373.0,NaN,NaN,NaN,WAKEFIELD AVENUE,BULLARD AVENUE,False,...,Unspecified,Unspecidifed,Unspecidifed,4297737,Sedan,Pick-up Truck,Sedan,Unspecified,Unspecified,"WAKEFIELD AVENUE,QUEENS,United States of America"
99982,2019-11-07T00:00:00.000,17:08,QUEENS,11373.0,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,BULLARD AVENUE,False,...,Unspecidifed,Unspecidifed,Unspecidifed,4237898,Station Wagon/Sport Utility Vehicle,Sedan,Unspecified,Unspecified,Unspecified,"BRONX WHITESTONE BRIDGE,QUEENS,United States o..."


,A,B,C,D
0,1,2,3,L1
4,2,3,4,5
1,4,5,6,L2
3,4,8,6,L2
2,7,8,9,L3
5,7,9,9,L3
